# On Combining Two Forecasts

Assume that we provide two probability distributions (one is yours one is mine) to foreacast two numbers.
The means are respectively (0,0) and (10,10). If a distribution has a higher variance (compared to the other distribution) that means that the owner of the distribution is less confident in her forecast (relative to the other).

First consider the situation that the **corellation is zero**, then:
1. If we are equally confident in our forecasts then the mean of the concencus distribution is (5,5).
2. If you're four times confident then me, then the mean of the concencus distribution is (2,2).

> ### What if the corolletion is different from zero?

In [2]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, Layout, HBox, VBox, Box, interactive_output,Label
from ipywidgets import Button
from scipy.stats import norm
import itertools
from scipy.stats import multivariate_normal


In [3]:
cartesian = pd.DataFrame()
i = 0
for element in itertools.product(*[[x for x in np.arange(-1,1,0.02)],[x for x in np.arange(-1,1,0.02)]]):
    i = i+1
    cartesian = cartesian.append(pd.DataFrame(data = list(element),columns=[i]).T)

In [9]:
def pdf_generator(first_cov11,first_cov22,corr1,first_mean1,first_mean2,sec_cov11,sec_cov22,corr2,sec_mean1,sec_mean2):
    first_cov12 = corr1*np.sqrt(first_cov11*first_cov22)
    first_cov = np.array([[first_cov11, first_cov12], [first_cov12, first_cov22]])
    
    min_eig = np.min(np.real(np.linalg.eigvals(first_cov)))
    if min_eig < 0:
        first_cov -= 10*min_eig * np.eye(*first_cov.shape)

    first_mean = np.array([first_mean1, first_mean2])
    
    sec_cov12 = corr2*np.sqrt(sec_cov11*sec_cov22)

    sec_cov = np.array([[sec_cov11, sec_cov12], [sec_cov12, sec_cov22]])
    
    min_eig = np.min(np.real(np.linalg.eigvals(sec_cov)))
    if min_eig < 0:
        sec_cov -= 10*min_eig * np.eye(*sec_cov.shape)
        
    sec_mean = np.array([sec_mean1, sec_mean2])
    
    pdf = cartesian.copy()
    y = multivariate_normal.pdf(pdf.values, mean=first_mean, cov=first_cov)
    z = multivariate_normal.pdf(pdf.values, mean=sec_mean, cov=sec_cov)
    pdf['Prior'] = y
    pdf['Prior'] = pdf['Prior']/(pdf['Prior'].sum())
    
    pdf['Likelihood'] = z
    pdf['Likelihood'] = pdf['Likelihood']/(pdf['Likelihood'].sum())
    
    pdf['Posterior'] = z*y
    pdf['Posterior'] = pdf['Posterior']/(pdf['Posterior'].sum()) 
  
    dist_prob_first = pdf.pivot(index=0,columns=1,values='Prior')
    dist_prob_sec = pdf.pivot(index=0,columns=1,values='Likelihood')
    dist_prob_multip = pdf.pivot(index=0,columns=1,values='Posterior')
    
    plt.figure(figsize=(15,8))
    prior = plt.contour(dist_prob_first.index, dist_prob_first.columns, dist_prob_first.values,5,cmap='coolwarm')
    plt.clabel(prior, inline=1, fontsize=16)
    likely = plt.contour(dist_prob_sec.index, dist_prob_sec.columns, dist_prob_sec.values,5,cmap='coolwarm')
    plt.clabel(likely, inline=1, fontsize=16)
    post = plt.contour(dist_prob_multip.index, dist_prob_multip.columns, dist_prob_multip.values,5,cmap='coolwarm')
    plt.clabel(post, inline=1, fontsize=16)
    plt.xlabel('x',fontsize=20)
    plt.ylabel('y',fontsize=20)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    
    
    #plt.colorbar()
    list_dist = ['Prior','Likelihood','Posterior']

    for i,k in zip(pdf[list_dist].idxmax().values,list_dist):
        plt.scatter(pdf.iloc[i][[1]],pdf.iloc[i][[0]],marker=(5,1,0),s=60,color='green')
        plt.text(pdf.iloc[i][[1]], pdf.iloc[i][[0]], k, fontsize=16)

    

In [10]:

first_cov11=widgets.FloatSlider(min = 0,max=0.10,step=0.005,value=0.01,continuous_update=False,description='Variance 1')
first_cov22=widgets.FloatSlider(min = 0,max=1,step=0.005,value=0.02,continuous_update=False,description='Variance 2')
corr1=widgets.FloatSlider(min = -1,max=1,step=0.1,value=0.01,continuous_update=False,description='Correlation')
first_mean1=widgets.FloatSlider(min = -2,max=3,step=0.1,value=-0.5,continuous_update=False,description='Mean 1')
first_mean2=widgets.FloatSlider(min = -2,max=3,step=0.1,value=-0.5,continuous_update=False,description='Mean 2')
sec_cov11=widgets.FloatSlider(min = 0,max=0.1,step=0.005,value = 0.01,continuous_update=False,description='Variance 1')
sec_cov22=widgets.FloatSlider(min = 0,max=0.1,step=0.005,value=0.02,continuous_update=False,description='Variance 2')
corr2=widgets.FloatSlider(min = -1,max=1,step=0.005,value=0.1,continuous_update=False,description='Correlation')
sec_mean1=widgets.FloatSlider(min = -1,max=1,step=0.1,value=0.5,continuous_update=False,description='Mean 1')
sec_mean2=widgets.FloatSlider(min = -1,max=1,step=0.1,value=0.5,continuous_update=False,description='Mean 2')

interactor = (interactive(pdf_generator,
                    first_cov11=first_cov11,
                    first_cov22=first_cov22,
                    corr1=corr1,
                    first_mean1=first_mean1,
                    first_mean2=first_mean2,
                    sec_cov11=sec_cov11,
                    sec_cov22=sec_cov22,
                    corr2=corr2,
                    sec_mean1=sec_mean1,
                   sec_mean2=sec_mean2
            ))


In [11]:
button1 = Button(description = 'Me',layout=Layout(width='30%'),button_style='danger')
button2 = Button(description = 'You',layout=Layout(width='30%'),button_style='danger')
header =HBox((button1, button2))
header.layout.justify_content = 'center'
display(header)

myh = HBox([VBox(interactor.children[:-1][:5]),VBox(interactor.children[:-1][5:])])
myh.layout.justify_content = 'center'
display(myh, interactor.children[-1])


Output()